# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2db672fc10>
├── 'a' --> tensor([[-1.3924, -0.0311,  1.1857],
│                   [-0.8526, -0.3420,  0.3321]])
└── 'x' --> <FastTreeValue 0x7f2db672f910>
    └── 'c' --> tensor([[ 1.1846, -1.7728, -0.7627, -0.1018],
                        [-0.2380, -0.5250,  1.5084, -0.4666],
                        [-0.5357, -0.1745,  0.2971, -1.6321]])

In [4]:
t.a

tensor([[-1.3924, -0.0311,  1.1857],
        [-0.8526, -0.3420,  0.3321]])

In [5]:
%timeit t.a

62.9 ns ± 0.0698 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2db672fc10>
├── 'a' --> tensor([[-1.7071, -1.0846, -0.0383],
│                   [-0.7532,  0.1694,  0.1776]])
└── 'x' --> <FastTreeValue 0x7f2db672f910>
    └── 'c' --> tensor([[ 1.1846, -1.7728, -0.7627, -0.1018],
                        [-0.2380, -0.5250,  1.5084, -0.4666],
                        [-0.5357, -0.1745,  0.2971, -1.6321]])

In [7]:
%timeit t.a = new_value

58.9 ns ± 0.0619 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.3924, -0.0311,  1.1857],
               [-0.8526, -0.3420,  0.3321]]),
    x: Batch(
           c: tensor([[ 1.1846, -1.7728, -0.7627, -0.1018],
                      [-0.2380, -0.5250,  1.5084, -0.4666],
                      [-0.5357, -0.1745,  0.2971, -1.6321]]),
       ),
)

In [10]:
b.a

tensor([[-1.3924, -0.0311,  1.1857],
        [-0.8526, -0.3420,  0.3321]])

In [11]:
%timeit b.a

62.3 ns ± 0.0157 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.6801, -2.1640,  0.4632],
               [ 0.8423, -0.3593, -0.3650]]),
    x: Batch(
           c: tensor([[ 1.1846, -1.7728, -0.7627, -0.1018],
                      [-0.2380, -0.5250,  1.5084, -0.4666],
                      [-0.5357, -0.1745,  0.2971, -1.6321]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 6.08 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

843 ns ± 0.36 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.8 µs ± 8.83 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 596 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 132 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2db6735730>
├── 'a' --> tensor([[[-1.3924, -0.0311,  1.1857],
│                    [-0.8526, -0.3420,  0.3321]],
│           
│                   [[-1.3924, -0.0311,  1.1857],
│                    [-0.8526, -0.3420,  0.3321]],
│           
│                   [[-1.3924, -0.0311,  1.1857],
│                    [-0.8526, -0.3420,  0.3321]],
│           
│                   [[-1.3924, -0.0311,  1.1857],
│                    [-0.8526, -0.3420,  0.3321]],
│           
│                   [[-1.3924, -0.0311,  1.1857],
│                    [-0.8526, -0.3420,  0.3321]],
│           
│                   [[-1.3924, -0.0311,  1.1857],
│                    [-0.8526, -0.3420,  0.3321]],
│           
│                   [[-1.3924, -0.0311,  1.1857],
│                    [-0.8526, -0.3420,  0.3321]],
│           
│                   [[-1.3924, -0.0311,  1.1857],
│                    [-0.8526, -0.3420,  0.3321]]])
└── 'x' --> <FastTreeValue 0x7f2db6735490>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.7 µs ± 67.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2d88b17cd0>
├── 'a' --> tensor([[-1.3924, -0.0311,  1.1857],
│                   [-0.8526, -0.3420,  0.3321],
│                   [-1.3924, -0.0311,  1.1857],
│                   [-0.8526, -0.3420,  0.3321],
│                   [-1.3924, -0.0311,  1.1857],
│                   [-0.8526, -0.3420,  0.3321],
│                   [-1.3924, -0.0311,  1.1857],
│                   [-0.8526, -0.3420,  0.3321],
│                   [-1.3924, -0.0311,  1.1857],
│                   [-0.8526, -0.3420,  0.3321],
│                   [-1.3924, -0.0311,  1.1857],
│                   [-0.8526, -0.3420,  0.3321],
│                   [-1.3924, -0.0311,  1.1857],
│                   [-0.8526, -0.3420,  0.3321],
│                   [-1.3924, -0.0311,  1.1857],
│                   [-0.8526, -0.3420,  0.3321]])
└── 'x' --> <FastTreeValue 0x7f2d88b17c10>
    └── 'c' --> tensor([[ 1.1846, -1.7728, -0.7627, -0.1018],
                        [-0.2380, -0.5250,  1.5084, -0.4666],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 20.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.5 µs ± 209 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.1846, -1.7728, -0.7627, -0.1018],
                       [-0.2380, -0.5250,  1.5084, -0.4666],
                       [-0.5357, -0.1745,  0.2971, -1.6321]],
              
                      [[ 1.1846, -1.7728, -0.7627, -0.1018],
                       [-0.2380, -0.5250,  1.5084, -0.4666],
                       [-0.5357, -0.1745,  0.2971, -1.6321]],
              
                      [[ 1.1846, -1.7728, -0.7627, -0.1018],
                       [-0.2380, -0.5250,  1.5084, -0.4666],
                       [-0.5357, -0.1745,  0.2971, -1.6321]],
              
                      [[ 1.1846, -1.7728, -0.7627, -0.1018],
                       [-0.2380, -0.5250,  1.5084, -0.4666],
                       [-0.5357, -0.1745,  0.2971, -1.6321]],
              
                      [[ 1.1846, -1.7728, -0.7627, -0.1018],
                       [-0.2380, -0.5250,  1.5084, -0.4666],
                       [-0.5357, -0.1745,  0.2971, -1.6321]],

In [26]:
%timeit Batch.stack(batches)

78.6 µs ± 137 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.1846, -1.7728, -0.7627, -0.1018],
                      [-0.2380, -0.5250,  1.5084, -0.4666],
                      [-0.5357, -0.1745,  0.2971, -1.6321],
                      [ 1.1846, -1.7728, -0.7627, -0.1018],
                      [-0.2380, -0.5250,  1.5084, -0.4666],
                      [-0.5357, -0.1745,  0.2971, -1.6321],
                      [ 1.1846, -1.7728, -0.7627, -0.1018],
                      [-0.2380, -0.5250,  1.5084, -0.4666],
                      [-0.5357, -0.1745,  0.2971, -1.6321],
                      [ 1.1846, -1.7728, -0.7627, -0.1018],
                      [-0.2380, -0.5250,  1.5084, -0.4666],
                      [-0.5357, -0.1745,  0.2971, -1.6321],
                      [ 1.1846, -1.7728, -0.7627, -0.1018],
                      [-0.2380, -0.5250,  1.5084, -0.4666],
                      [-0.5357, -0.1745,  0.2971, -1.6321],
                      [ 1.1846, -1.7728, -0.7627, -0.1018],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 585 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

305 µs ± 1.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
